<a href="https://colab.research.google.com/github/TerrorismAnalyticsBureau/TAB-AI/blob/main/TABAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import os
import chardet
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Set pyTorch local env to use segmented GPU memory
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear GPU cache & Set the device to use GPU
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
# Skip rows = 1 because those are the column names
X = np.array([])


# Load the data
# Open the file and detect the encoding
#with open('./globalterrorismdb_0718dist.csv', 'rb') as f:
#    result = chardet.detect(f.read())

# Get the detected encoding
#print(f"Detected encoding: {result['encoding']}")

# Now, read the file using the detected encoding
data = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding="Windows-1252")

# Extract relevant columns (adjust indices or column names as needed)
input_columns = data.iloc[:, [1, 2, 3, 7, 11]]
input_columns = input_columns.fillna(0)

# Convert non-numeric to numeric and fill missing values
for col in input_columns.columns:
    input_columns[col] = pd.to_numeric(input_columns[col], errors='coerce')  # Convert non-numeric to NaN
input_columns = input_columns.fillna(0)  # Replace NaN with 0

# Normalize the first three columns
scaler = MinMaxScaler()
input_columns.iloc[:, :3] = scaler.fit_transform(input_columns.iloc[:, :3].astype(float))

attack_target = data.iloc[:, [28]]
group_target = data.iloc[:, [57]]
date_target = data.iloc[:, [1, 2, 3]]

# Extract unique group names
unique_groups = list(set(data['gname']))
unique_provstates = list(set(data['provstate']))
unique_cities = list(set(data['city']))

# Initialize LabelEncoder and fit to the unique groups
group_encoder = LabelEncoder()
group_encoder.fit(unique_groups)

provstate_encoder = LabelEncoder()
provstate_encoder.fit(unique_provstates)

city_encoder = LabelEncoder()
city_encoder.fit(unique_cities)

# Create a dictionary to map names to their encoded IDs
group_dict = pd.Series(group_encoder.transform(unique_groups), index=unique_groups)
provstate_dict = pd.Series(provstate_encoder.transform(unique_provstates), index=unique_provstates)
city_dict = pd.Series(city_encoder.transform(unique_cities), index=unique_cities)

# Assign values to tensors for processing
input_tensor = torch.tensor(input_columns.to_numpy(), dtype=torch.float32)
attack_target_tensor = torch.tensor(attack_target.values, dtype=torch.float32)
group_target_tensor = torch.tensor(group_encoder.fit_transform(group_target.values), dtype=torch.float32)
date_target_tensor = torch.tensor(date_target.values, dtype=torch.float32)

# TESTING - PRINT DICTIONARY ITEMS
#for key, value in group_dict.items():
#  print("group: ", key, "| ID #:", value)

#for key, value in provstate_dict.items():
#  print("provstate: ", key, "| ID #:", value)

#for key, value in city_dict.items():
#  print("city: ", key, "| ID #:", value)

# Assign values to tensors for processing
X_tensor = input_tensor
Y_tensor_attack = attack_target_tensor
Y_tensor_group = group_target_tensor
Y_tensor_date = date_target_tensor

# Set tensors to use GPU
X_tensor = X_tensor.to(device)
Y_tensor_attack = Y_tensor_attack.to(device)
Y_tensor_group = Y_tensor_group.to(device)
Y_tensor_date = Y_tensor_date.to(device)

class LSTMAttackPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMAttackPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out)  # Dropout after LSTM
        logits = self.fc(lstm_out[:, -1, :])  # Fully connected on the last time step
        return logits  # Return raw logits for training with CrossEntropyLoss

sequence_length = 10

# Create sequences from the input_tensor
def create_sequences(input_data, seq_length):
    sequences = []
    for i in range(len(input_data) - seq_length + 1):
        seq = input_data[i:i+seq_length]
        sequences.append(seq)
    return torch.stack(sequences)

# Assuming input_tensor is of shape (num_samples, num_features)
sequences = create_sequences(X_tensor, sequence_length)
print(sequences.shape)  # (num_sequences, sequence_length, num_features)

batch_size = 32

# Assuming you also have a target tensor (e.g., attack_target_tensor)
dataset = TensorDataset(sequences, Y_tensor_attack[:len(sequences)])  # Adjust target size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = LSTMAttackPredictor(input_size=input_tensor.shape[1], hidden_size=128, output_size=3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()  # Assuming classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Training loop
for epoch in range(10):  # Number of epochs
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)  # Move to GPU if available

        # Forward pass
        outputs = model(batch_x)

        # Compute loss
        loss = criterion(outputs, batch_y.long())

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")






RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


name (csv column number)
```
•    iyear (1), imonth (2), iday (3)
•    country code (7) and country_txt (8)
•    region code (9) and region_txt (10)
•    provstate (11) and city (12)
•    latitude (13) and longitude (14)
•    attacktype1 (28) and attacktype1_txt (29)
•    targtype1 (34) and targtype1_txt (35)
•    targsubtype1 (36) and targsubtype1_txt (37)
•    target1 (39) (the specific target by name, building or person)

•    natity1 (40) and natity1_txt (41) (maybe later)
•    gname (group name) (57)
•    weaptype1 (81) and weaptype1_txt (82) (maybe)
•    nkill (98) and nwound (101)

```

